In [1]:
# from sst and rainfall ---> rainfall
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np
from netCDF4 import Dataset
import matplotlib.pyplot as plt 
import tensorflow.compat.v1 as tf
import keras.backend as K
from keras.models import Sequential
from keras.layers import Conv3D, Conv2D
from keras.layers import ConvLSTM2D
from keras.layers import Attention
from keras.layers import Input
from keras.models import Model
from keras.layers import Dropout, Lambda
from keras import optimizers
from keras.layers import Conv2DTranspose, TimeDistributed
from keras import callbacks
import xarray as xr

# Define global variables
lead_time = 0
save_path = "/home/cccr/roxy/matin/MTech_project/model/Conv-LSTM/7in1out/"
model_path = save_path + "ConvLstm_sst0rf_rf_0SI.h5"
add_data = "/home/cccr/roxy/matin/MTech_project/data/"

def preprocess_data(sequence, n_steps, channels):
    """
    Function to preprocess the data and prepare it for model training.
    
    Parameters:
        sequence (xarray.Dataset): Data sequence to be preprocessed.
        n_steps (int): Number of time steps in the input sequence.
        channels (int): Number of channels in the input sequence.
    
    Returns:
        tuple: A tuple containing the preprocessed input sequence (X) and output sequence (y).
    """
    # Select data for Jun, Jul, Aug, Sept
    sequence = sequence.sel(time=slice("1998-01-01","2016-12-31"))
    sequence = sequence.where(sequence.time.dt.month.isin([6,7,8,9]), drop=True)
    max = sequence.max()
    min = sequence.min()
    
    # Max-min normalization
    sequence = (sequence-min)/(max - min)
    sequence = np.array(sequence)
    
    # Exponential Space Transform
    valid_pts = np.where(sequence==sequence)
    sequence[valid_pts] = np.exp(sequence[valid_pts])
    sequence[valid_pts] = np.power(sequence[valid_pts],7)
    invalid_pts = np.where(sequence != sequence)
    sequence[invalid_pts] = 0
    
    # Prepare input and output sequences
    X, y = [], []
    for i in range(len(sequence)):
        end_ix = i + n_steps
        if end_ix + lead_time > len(sequence)-1:
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix + lead_time]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)


def create_input(channels, n_steps):
    """
    Function to create the input for the model.
    
    Parameters:
        channels (list): List of channels to be used in the input.
        n_steps (int): Number of time steps in the input sequence.
        
    Returns:
        numpy.ndarray: The input for the model.
    """
    stack = []
    for i in range(len(channels)):
        channel_data = xr.open_dataarray(f"/home/cccr/roxy/matin/MTech_project/data/{channels[i]}")
        input, _ = preprocess_data(channel_data, n_steps=n_steps, channels=channels[i])
        input = np.expand_dims(input, axis=2)
        print(f"Adding channel {i} with shape: {input.shape}")
        stack.append(input)
    out = np.dstack(stack)
    return out



channels = ["SIFilteredrfBOB_0lag.nc","SIFilteredSSTBOB_0.nc"]
steps = 7

# Create input for the model
input = create_input(channels, steps)

# Preprocess target data
channel_data = xr.open_dataarray(add_data + "SIFilteredrfBOB_0lag.nc")
_, target = preprocess_data(channel_data, n_steps=steps, channels=channels[-1])
target = np.expand_dims(target, axis=1)

# Roll axes for input and target
x = np.rollaxis(input, 4, 2)
x = np.rollaxis(x, 4, 2)

y = np.rollaxis(target, 3, 1)
y = np.rollaxis(y, 3, 1)

# Assert that input and target shapes are consistent
assert input.shape[0] == target.shape[0]
assert input.shape[-1] == target.shape[-1]
assert input.shape[-2] == target.shape[-2]


# Print shape of input and target
print("INPUT SHAPE --> ", input.shape)
print("TARGET SHAPE --> ", target.shape)

# Delete original input and target variables to free up memory
del input
del target

# Perform additional processing or modeling steps as needed
seq = tf.keras.Sequential()

# Add layers to the model
seq.add(ConvLSTM2D(filters=4, kernel_size=(3,3), padding='same', input_shape=(7,25,39,2), return_sequences=True, data_format='channels_last'))
seq.add(ConvLSTM2D(filters=8, kernel_size=(3,3), padding='same', return_sequences=True, data_format='channels_last'))
seq.add(ConvLSTM2D(filters=8, kernel_size=(3,3), padding='same', return_sequences=True, data_format='channels_last'))
seq.add(ConvLSTM2D(filters=16, kernel_size=(3,3), padding='same', return_sequences=True, data_format='channels_last'))
seq.add(ConvLSTM2D(filters=16, kernel_size=(3,3), padding='same', return_sequences=False, data_format='channels_last'))
seq.add(Conv2D(filters=15, kernel_size=(3,3), activation='relu', padding='same', data_format='channels_last'))
seq.add(Conv2D(filters=1, kernel_size=(3,3), activation='relu', padding='same', data_format='channels_last'))
# Create the query, value, and key inputs
query_input = Input(shape=(25,39,2), name='query_input')
value_input = Input(shape=(25,39,2), name='value_input')
key_input = Input(shape=(25,39,2), name='key_input')

# Pass the inputs to the Attention layer
attention_output = Attention(name='attention')([query_input, value_input, key_input])
Adam = tf.keras.optimizers.Adam(learning_rate=10**-4)
seq.compile(loss='mean_squared_error', optimizer=Adam, metrics=['mae'])

print(seq.summary())

# Define the number of training epochs
n_epochs = 300
logdir = save_path
# Define the TensorBoard callback for monitoring training progress
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=10, batch_size=10, write_graph=True)

# Define the ReduceLROnPlateau callback for reducing the learning rate when the model plateaus
lr_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=20, min_lr=10**-20)

# Define the ModelCheckpoint callback for saving the best model based on validation mean absolute error
checkpoint_callback_mae = tf.keras.callbacks.ModelCheckpoint(model_path, monitor='val_mae',
                                                             mode='min', save_best_only=True)

# Define the ModelCheckpoint callback for saving the best model based on validation loss
checkpoint_callback_loss = tf.keras.callbacks.ModelCheckpoint(model_path, monitor='val_loss',
                                                              mode='min', save_best_only=True)

# Define the EarlyStopping callback for stopping training when the model stops improving
early_stop_callback = tf.keras.callbacks.EarlyStopping(patience=30, restore_best_weights=True)

# Define the TerminateOnNaN callback for terminating the training process if there is NaN value in any of the output
terminate_callback = tf.keras.callbacks.TerminateOnNaN()

# Train the model using the defined callbacks
history = seq.fit(x, y, epochs=n_epochs, validation_split=0.1,
                  batch_size=50, callbacks=[early_stop_callback,
                  checkpoint_callback_mae, checkpoint_callback_loss,
                  tb_callback, lr_callback, terminate_callback])
#Save the trained model
seq.save(model_path)

#Save the training history

np.save(f'{save_path}ConvLstm_sst0rf_rf_0SI.npy', history.history)               
              

Adding channel 0 with shape: (2311, 7, 1, 25, 39)
Adding channel 1 with shape: (2311, 7, 1, 25, 39)
INPUT SHAPE -->  (2311, 7, 2, 25, 39)
TARGET SHAPE -->  (2311, 1, 25, 39)


2023-03-21 11:06:09.316403: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_lstm2d (ConvLSTM2D)    (None, 7, 25, 39, 4)      880       
                                                                 
 conv_lstm2d_1 (ConvLSTM2D)  (None, 7, 25, 39, 8)      3488      
                                                                 
 conv_lstm2d_2 (ConvLSTM2D)  (None, 7, 25, 39, 8)      4640      
                                                                 
 conv_lstm2d_3 (ConvLSTM2D)  (None, 7, 25, 39, 16)     13888     
                                                                 
 conv_lstm2d_4 (ConvLSTM2D)  (None, 25, 39, 16)        18496     
                                                                 
 conv2d (Conv2D)             (None, 25, 39, 15)        2175      
                                                                 
 conv2d_1 (Conv2D)           (None, 25, 39, 1)         1

42/42 [==============================] - 38s 911ms/step - loss: 165.4832 - mae: 7.2846 - val_loss: 96.6151 - val_mae: 6.3706 - lr: 1.0000e-04
Epoch 45/300
42/42 [==============================] - 36s 867ms/step - loss: 163.2008 - mae: 7.2302 - val_loss: 96.9712 - val_mae: 6.6042 - lr: 1.0000e-04
Epoch 46/300
42/42 [==============================] - 45s 1s/step - loss: 159.5351 - mae: 7.1276 - val_loss: 92.0772 - val_mae: 6.2450 - lr: 1.0000e-04
Epoch 47/300
42/42 [==============================] - 40s 962ms/step - loss: 157.1793 - mae: 7.0675 - val_loss: 89.9218 - val_mae: 6.2622 - lr: 1.0000e-04
Epoch 48/300
42/42 [==============================] - 37s 891ms/step - loss: 154.0591 - mae: 6.9760 - val_loss: 87.9003 - val_mae: 6.1416 - lr: 1.0000e-04
Epoch 49/300
42/42 [==============================] - 39s 933ms/step - loss: 151.4593 - mae: 6.9101 - val_loss: 85.0844 - val_mae: 5.9887 - lr: 1.0000e-04
Epoch 50/300
42/42 [==============================] - 47s 1s/step - loss: 148.9910 - m

42/42 [==============================] - 42s 991ms/step - loss: 50.7554 - mae: 3.4023 - val_loss: 21.6889 - val_mae: 2.9737 - lr: 1.0000e-04
Epoch 98/300
42/42 [==============================] - 42s 1s/step - loss: 49.9688 - mae: 3.3701 - val_loss: 20.6439 - val_mae: 2.9184 - lr: 1.0000e-04
Epoch 99/300
42/42 [==============================] - 42s 1s/step - loss: 49.0570 - mae: 3.3230 - val_loss: 20.1587 - val_mae: 2.8797 - lr: 1.0000e-04
Epoch 100/300
42/42 [==============================] - 41s 977ms/step - loss: 48.5286 - mae: 3.2998 - val_loss: 20.0854 - val_mae: 2.8709 - lr: 1.0000e-04
Epoch 101/300
42/42 [==============================] - 41s 985ms/step - loss: 47.8884 - mae: 3.2781 - val_loss: 20.0222 - val_mae: 2.8636 - lr: 1.0000e-04
Epoch 102/300
42/42 [==============================] - 39s 920ms/step - loss: 47.2678 - mae: 3.2399 - val_loss: 19.4305 - val_mae: 2.8115 - lr: 1.0000e-04
Epoch 103/300
42/42 [==============================] - 42s 997ms/step - loss: 46.6457 - mae:

42/42 [==============================] - 43s 1s/step - loss: 31.8746 - mae: 2.5975 - val_loss: 12.1159 - val_mae: 2.1655 - lr: 1.0000e-04
Epoch 151/300
42/42 [==============================] - 43s 1s/step - loss: 31.2692 - mae: 2.5455 - val_loss: 11.7203 - val_mae: 2.1328 - lr: 1.0000e-04
Epoch 152/300
42/42 [==============================] - 45s 1s/step - loss: 30.9260 - mae: 2.5087 - val_loss: 11.5809 - val_mae: 2.1067 - lr: 1.0000e-04
Epoch 153/300
42/42 [==============================] - 38s 917ms/step - loss: 30.6954 - mae: 2.5109 - val_loss: 11.5030 - val_mae: 2.0959 - lr: 1.0000e-04
Epoch 154/300
42/42 [==============================] - 46s 1s/step - loss: 30.9298 - mae: 2.5580 - val_loss: 11.3920 - val_mae: 2.0863 - lr: 1.0000e-04
Epoch 155/300
42/42 [==============================] - 41s 975ms/step - loss: 30.3455 - mae: 2.4943 - val_loss: 11.4019 - val_mae: 2.0875 - lr: 1.0000e-04
Epoch 156/300
42/42 [==============================] - 39s 926ms/step - loss: 30.1533 - mae: 2.4

42/42 [==============================] - 40s 965ms/step - loss: 23.0255 - mae: 2.2561 - val_loss: 9.4653 - val_mae: 1.8990 - lr: 1.0000e-04
Epoch 204/300
42/42 [==============================] - 43s 1s/step - loss: 23.0792 - mae: 2.2699 - val_loss: 9.2699 - val_mae: 1.8805 - lr: 1.0000e-04
Epoch 205/300
42/42 [==============================] - 43s 1s/step - loss: 22.8703 - mae: 2.2519 - val_loss: 9.5952 - val_mae: 1.9128 - lr: 1.0000e-04
Epoch 206/300
42/42 [==============================] - 42s 1s/step - loss: 22.7281 - mae: 2.2407 - val_loss: 9.3176 - val_mae: 1.8979 - lr: 1.0000e-04
Epoch 207/300
42/42 [==============================] - 39s 920ms/step - loss: 22.5315 - mae: 2.2385 - val_loss: 9.3412 - val_mae: 1.8974 - lr: 1.0000e-04
Epoch 208/300
42/42 [==============================] - 40s 946ms/step - loss: 22.5576 - mae: 2.2536 - val_loss: 9.6841 - val_mae: 1.9986 - lr: 1.0000e-04
Epoch 209/300
42/42 [==============================] - 40s 954ms/step - loss: 22.7798 - mae: 2.2919

Epoch 257/300
42/42 [==============================] - 38s 899ms/step - loss: 18.2033 - mae: 2.1327 - val_loss: 8.2115 - val_mae: 1.7701 - lr: 1.0000e-04
Epoch 258/300
42/42 [==============================] - 39s 934ms/step - loss: 18.0395 - mae: 2.1001 - val_loss: 8.1303 - val_mae: 1.7585 - lr: 1.0000e-04
Epoch 259/300
42/42 [==============================] - 40s 954ms/step - loss: 18.0946 - mae: 2.1299 - val_loss: 8.6559 - val_mae: 1.9249 - lr: 1.0000e-04
Epoch 260/300
42/42 [==============================] - 35s 846ms/step - loss: 17.9496 - mae: 2.1193 - val_loss: 8.6850 - val_mae: 1.8594 - lr: 1.0000e-04
Epoch 261/300
42/42 [==============================] - 47s 1s/step - loss: 18.5294 - mae: 2.1400 - val_loss: 8.0623 - val_mae: 1.7617 - lr: 1.0000e-04
Epoch 262/300
42/42 [==============================] - 44s 1s/step - loss: 18.2876 - mae: 2.1392 - val_loss: 8.6006 - val_mae: 1.8669 - lr: 1.0000e-04
Epoch 263/300
42/42 [==============================] - 41s 986ms/step - loss: 17.7

In [2]:
def preprocess_data(sequence, n_steps, channels):
    """
    Function to preprocess the data and prepare it for model training.
    
    Parameters:
        sequence (xarray.Dataset): Data sequence to be preprocessed.
        n_steps (int): Number of time steps in the input sequence.
        channels (int): Number of channels in the input sequence.
    
    Returns:
        tuple: A tuple containing the preprocessed input sequence (X) and output sequence (y).
    """
    # Select data for Jun, Jul, Aug, Sept
    sequence = sequence.sel(time=slice("2017-01-01","2018-12-31"))
    sequence = sequence.where(sequence.time.dt.month.isin([6,7,8,9]), drop=True)
    max = sequence.max()
    min = sequence.min()
    
    # Max-min normalization
    sequence = (sequence-min)/(max - min)
    sequence = np.array(sequence)
    
    # Exponential Space Transform
    valid_pts = np.where(sequence==sequence)
    sequence[valid_pts] = np.exp(sequence[valid_pts])
    sequence[valid_pts] = np.power(sequence[valid_pts],7)
    invalid_pts = np.where(sequence != sequence)
    sequence[invalid_pts] = 0
    
    # Prepare input and output sequences
    X, y = [], []
    for i in range(len(sequence)):
        end_ix = i + n_steps
        if end_ix + lead_time > len(sequence)-1:
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix + lead_time]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)


def create_input(channels, n_steps):
    """
    Function to create the input for the model.
    
    Parameters:
        channels (list): List of channels to be used in the input.
        n_steps (int): Number of time steps in the input sequence.
        
    Returns:
        numpy.ndarray: The input for the model.
    """
    stack = []
    for i in range(len(channels)):
        channel_data = xr.open_dataarray(f"/home/cccr/roxy/matin/MTech_project/data/{channels[i]}")
        input, _ = preprocess_data(channel_data, n_steps=n_steps, channels=channels[i])
        input = np.expand_dims(input, axis=2)
        print(f"Adding channel {i} with shape: {input.shape}")
        stack.append(input)
    out = np.dstack(stack)
    return out



channels = ["SIFilteredrfBOB_0lag.nc","SIFilteredSSTBOB_0.nc"]
steps = 7

# Create input for the model
input = create_input(channels, steps)

# Preprocess target data
channel_data = xr.open_dataarray(add_data + "SIFilteredrfBOB_0lag.nc")
_, target = preprocess_data(channel_data, n_steps=steps, channels=channels[-1])
target = np.expand_dims(target, axis=1)

# Roll axes for input and target
x = np.rollaxis(input, 4, 2)
x = np.rollaxis(x, 4, 2)

y = np.rollaxis(target, 3, 1)
y = np.rollaxis(y, 3, 1)

# Assert that input and target shapes are consistent
assert input.shape[0] == target.shape[0]
assert input.shape[-1] == target.shape[-1]
assert input.shape[-2] == target.shape[-2]


# Print shape of input and target
print("INPUT SHAPE --> ", input.shape)
print("TARGET SHAPE --> ", target.shape)

# Delete original input and target variables to free up memory
del input
del target



Adding channel 0 with shape: (237, 7, 1, 25, 39)
Adding channel 1 with shape: (237, 7, 1, 25, 39)
INPUT SHAPE -->  (237, 7, 2, 25, 39)
TARGET SHAPE -->  (237, 1, 25, 39)


In [3]:
model = tf.keras.models.load_model(model_path)
pred = model.predict(x)

actual = y.flatten()
prediction = pred.flatten()
corr = np.corrcoef(actual,prediction)
corr

8/8 [==============================] - 3s 99ms/step


array([[1.        , 0.87096774],
       [0.87096774, 1.        ]])